In [1]:
import numpy as np
import pandas as pd

In [6]:
importance = pd.read_csv('/code/models/lgb_proceeded_train_20200116_102618_20200117_211418/all_importance.csv',index_col=0)

In [52]:
importance.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14
Clip,104.950459,233.659678,244.707052,92.969399,206.109221,24.244580,108.923098,116.309650,276.734766,559.172697,67.050799,103.677380,262.46972,302.479903,91.123352
Activity,0.000000,0.000000,0.000000,14.887500,0.000000,11.404300,13.463950,23.234700,11.594700,0.000000,16.805401,24.602400,0.00000,7.026090,0.000000
Assessment,10.862900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.591000,0.000000,11.039100,9.42054,0.000000,0.000000
Game,0.000000,0.000000,0.000000,45.513901,0.000000,0.000000,9.075070,6.515460,0.000000,8.237450,0.000000,7.837030,22.08500,11.627400,10.185000
session_title,2088.256493,3176.695627,1779.851142,2918.275608,1232.314510,1381.710487,1313.295803,1330.236687,2464.587150,2942.818735,1118.564796,2490.029232,2260.09873,3225.542613,18.380671


In [77]:
importance.mean(axis=1).sort_values(ascending=False)

session_title                                          1982.710552
label_session_title_count_action_label                 1392.422898
count_accuracy                                          578.623875
count_label_session_title_mean_accuracy_group_label     379.533877
count_label_session_title_count_accuracy_label          326.937477
                                                          ...     
d06f75b5                                                  0.000000
13f56524                                                  0.000000
65abac75                                                  0.000000
47f43a44                                                  0.000000
27253bdc                                                  0.000000
Length: 700, dtype: float64

In [102]:
train = pd.read_csv('/code/data/processed/proceeded_train_20200116_102618.csv', index_col=0)
test = pd.read_csv('/code/data/processed/proceeded_test_20200116_102618.csv', index_col=0)

In [103]:
class Preprocess:

    def process(self, df):
        df = self.common_process(df, 'date', lank=3)
        df = self.common_process(df, 'hour', lank=3)
        df = self.common_process(df, 'month', lank=1)
        df = self.common_process(df, 'dayofweek', lank=1)
        return df

    def common_process(self, df, word_in_colums: str, lank: int):
        columns = df.columns[df.columns.str.contains(word_in_colums)]
        df[f'mean_{word_in_colums}_count'] = df[columns].mean(axis=1)
        df[f'std_{word_in_colums}_count'] = df[columns].std(axis=1)
        df[f'non_zero_{word_in_colums}'] = (df[columns] == 0).sum(axis=1)
        new_columns = [f'{word_in_colums}_lank{i}' for i in range(lank)]
        print(new_columns)
        df[new_columns] = self.best_columns(df=df, lank=lank, columns=columns)
        df.drop(columns, axis=1, inplace=True)
        return df

    def best_columns(self, df, lank, columns):
        df = df[columns]
        rename_columns = {date: i for i, date in enumerate(columns)}
        df = df.rename(columns=rename_columns)
        best_df = df.rank(axis=1, ascending=False).apply(
            lambda x: pd.Series(list(x.sort_values()[0:lank].index)), axis=1
            )
        return best_df

In [104]:
preprocess = Preprocess()

In [105]:
train = preprocess.process(train)

['date_lank0', 'date_lank1', 'date_lank2']
['hour_lank0', 'hour_lank1', 'hour_lank2']
['month_lank0']
['dayofweek_lank0']


In [106]:
test = preprocess.process(test)

['date_lank0', 'date_lank1', 'date_lank2']
['hour_lank0', 'hour_lank1', 'hour_lank2']
['month_lank0']
['dayofweek_lank0']


In [107]:
train.head()

,Clip,Activity,Assessment,Game,session_title,count_correct_attempts,count_uncorrect_attempts,count_accuracy,acc_Cart Balancer (Assessment),acc_Cauldron Filler (Assessment),...,hour_lank1,hour_lank2,mean_month_count,std_month_count,non_zero_month,month_lank0,mean_dayofweek_count,std_dayofweek_count,non_zero_dayofweek,dayofweek_lank0
0,11,3,0,4,42,0,0,0.0,-1.0,-1.0,...,4,0,161.75,323.5,3,1,92.428571,244.543014,6,1
1,14,4,1,6,30,1,0,1.0,-1.0,-1.0,...,4,0,285.75,571.5,3,1,163.285714,432.013393,6,1
2,14,4,2,6,42,1,11,0.5,-1.0,-1.0,...,4,0,307.50,615.0,3,1,175.714286,464.896302,6,1
3,24,9,4,10,42,2,11,0.5,-1.0,-1.0,...,17,20,539.75,1079.5,3,1,308.428571,816.025297,6,1
4,28,10,5,13,30,3,12,0.5,-1.0,-1.0,...,20,17,646.50,1293.0,3,1,369.428571,977.416127,6,1


In [108]:
test.head()

,Clip,Activity,Assessment,Game,session_title,count_correct_attempts,count_uncorrect_attempts,count_accuracy,acc_Cart Balancer (Assessment),acc_Cauldron Filler (Assessment),...,hour_lank1,hour_lank2,mean_month_count,std_month_count,non_zero_month,month_lank0,mean_dayofweek_count,std_dayofweek_count,non_zero_dayofweek,dayofweek_lank0
0,14,7,1,3,19,1,0,1.000000,1.0,-1.0,...,16,18,216.75,433.500000,3,2,123.857143,168.179892,4,3
1,29,11,5,12,13,4,7,0.466667,1.0,0.5,...,22,20,679.50,1315.368009,2,2,388.285714,998.507171,5,6
2,6,2,0,0,42,0,0,0.000000,-1.0,-1.0,...,23,2,37.25,73.834838,2,2,21.285714,55.876991,5,5
3,10,2,0,1,42,0,0,0.000000,-1.0,-1.0,...,17,6,58.25,116.500000,3,0,33.285714,87.185080,4,5
4,17,1,0,6,13,0,0,0.000000,-1.0,-1.0,...,15,17,237.75,278.795475,2,1,135.857143,233.033208,3,3


In [109]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    np.random.seed(seed)
    # ラベルの数をカウント
    labels_num = np.max(y) + 1
    # 各グループのラベルの数をカウントする
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1
    # 各フォールドのラベルの数をカウント
    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)

    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)

    for i in range(k):
        test_k = i
        # val_k = i+1 if i+1 != k else 0
        # print(val_k)
        train_groups = all_groups - groups_per_fold[test_k]  #  - groups_per_fold[val_k]
        # val_groups = groups_per_fold[val_k]
        test_groups = groups_per_fold[test_k]
        # print(test_groups)
        # train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        # val_indices = [i for i, g in enumerate(groups) if g in val_groups]
        # test_indices = {str(g): [i for i, g in enumerate(groups) if g in test_groups]}

        def choice_ind(group):
            indices = []
            n_g = None
            list_ = []
            for i, g in enumerate(groups):
                if g in group:
                    if n_g is not None and n_g != g:
                        indices.append(list_)
                        list_ = []
                    list_.append(i)
                    n_g = g

            indices = [np.random.choice(i) for i in indices]
            return indices

        train_indices = choice_ind(train_groups)
        test_indices = choice_ind(test_groups)
        # print(train_indices)
        yield train_indices, test_indices  # val_indices,

In [110]:
FEATURES_FOR_TARGET_ENCODING = [
    # 'frequency_label',
    # 'count_action_label',
    # 'count_accuracy_label',
    'mean_accuracy_group_label',
    # 'label_description_val',
    # 'label_session_title_count_accuracy_label',
    # 'label_mean_accuracy_group_label_description_val',
    # 'label_frequency_label_description_val'
    ]

LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'rmse',
    }


def lgb_model(x, y, train_ind, test_ind, test_x, seed):
    x_train = x.iloc[train_ind]
    y_train = y.iloc[train_ind]
    x_test = x.iloc[test_ind]
    y_test = y.iloc[test_ind]

    target_encoding = TargetEncoding()
    x_train, x_test, test_x = target_encoding.process(
        x_train, y_train, x_test, test_x,
        columns=FEATURES_FOR_TARGET_ENCODING,
        seed=seed)

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_val = lgb.Dataset(x_test, y_test, reference=lgb_train)

    model = lgb.train(params=LGB_PARAMS,
                      train_set=lgb_train,
                      valid_sets=lgb_val,
                      feval=lgb_qwk,
                      num_boost_round=1000,
                      early_stopping_rounds=50)

    y_pred = model.predict(x_test, num_iteration=model.best_iteration)

    importance = pd.DataFrame(model.feature_importance('gain'),
                              index=x_train.columns)
    return model, y_pred, importance